### Import libraries and data

In [ ]:
import pandas as pd
import csv
import re
import numpy as np

In [ ]:
vols = pd.read_csv("volumes.csv")
vols = vols.drop("Unnamed: 0", axis=1)

In [ ]:
temps = pd.read_csv("temperatures.csv")
temps = temps.drop("Unnamed: 0", axis=1)

In [ ]:
unitformula = pd.read_csv("scaledformulae.csv")
unitformula = unitformula.drop("Unnamed: 0", axis=1)

### Setting models

In [ ]:
from sklearn import linear_model, svm, kernel_ridge
from scipy.optimize import nnls 
from sklearn.metrics import mean_squared_error
regVols = linear_model.LinearRegression()
regAlpha = linear_model.LinearRegression()
#regVols = linear_model.ElasticNet(positive = True)

### RMSD function

In [ ]:
def getRMSD (y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

### Percentage function

In [ ]:
def percCalc (row):
    predictedvolume = row["sum"]
    vols = row["Volume"]
    percentage = predictedvolume / vols
    if (percentage >=0.95) and (percentage <=1.05):
        return 1
    else:
        return 0

### Loop algorithm

In [ ]:
alpha = 0.00
elements = unitformula.columns.values
unitformula["Keep"] = 1
weights = []
alphas =[]
for chunk in zip(unitformula.groupby(np.arange(len(unitformula)) // 10000),vols.groupby(np.arange(len(vols)) // 10000),temps.groupby(np.arange(len(temps)) // 10000)):
    print "Chunk: ", chunk[0][0]
    prevalpha = 1
    RMSD = 10
    chunkformula = chunk[0][1].reset_index(drop=True)
    chunkvol = chunk[1][1].reset_index(drop=True)
    chunktemp = chunk[2][1].reset_index(drop=True)
    keepidx = np.arange(len(chunkformula))
    i = 0
    while abs(prevalpha-alpha) > 0.00001 or (RMSD > 1) :
        try:
            print "Iteration:", i
            #regVols.fit(chunkformula[chunkformula.Keep == 1], chunkvol.loc[keepidx].div(chunktemp.loc[keepidx].Temperature*alpha+1, axis="index"))
            if i > 0:
                prevw = w#[0]
                #print "old w: ", prevw
            #w = regVols.coef_[0]
            w, rnorm = nnls(np.matrix(chunkformula[chunkformula.Keep == 1]),
                            np.squeeze(np.array(chunkvol.iloc[keepidx].div(chunktemp.iloc[keepidx].Temperature*alpha+1, axis="index"))))
            if i > 0:
                RMSD = getRMSD(prevw,w)#[0]
                #print "New w", w
                print "RMSD: ", RMSD
            x2=pd.DataFrame()
            for line in zip(elements, w): #[0]
                x2[line[0]] = chunk[0][1][line[0]].apply(lambda x: x*line[1])
            x2["sum"] = x2.sum(axis=1)
            #if i>0:
                #print x2["sum"]
            vol2 = chunkvol.iloc[keepidx].div(x2["sum"].iloc[keepidx], axis="index") - 1
            #regAlpha.fit(chunktemp.loc[keepidx], vol2)
            if i > 0:
                prevalpha = alpha
                print "Previous Alpha: ", prevalpha
            #alpha = regAlpha.coef_[0][0]
            alpha, rnorm2 = nnls(np.matrix(chunktemp.iloc[keepidx]),np.squeeze(np.array(vol2)))
            print "Alpha: ", alpha
            print "deltaAlpha: ", abs(prevalpha-alpha)
            #print "dAlpha: ", abs(prevalpha - alpha)
            chunkformula["Keep"] = pd.concat([x2,vols], axis=1).apply(percCalc, axis=1)
            keepidx = chunkformula.Keep[chunkformula.Keep == 1].index.tolist()
            #print "Element average volume (cubic angstroms)", zip(elements,w[0])
            if (len(keepidx) <= 1) or (i > 200):
                print "wasted ur time bro"
                break
        except ValueError:
            break
        i =+ 1
    weights.append(w)#[0])
    alphas.append(alpha)

This loop is similar to the previous one; however now the the modelling technique is NNLS; the linear regression has been commented out, just as a test to check how well the chunk loop worked with NNLS.

Due to an error the volumes had to be squeezed due to having a array of shape length 2 (2D), rather than 1.
This problem was also exhibited in the alpha calculation; where vol2 had to be squeezed to 1D.

### Misc.

In [ ]:
print np.where(np.isinf(vol2))

Due to some problems, vol2 was searched for any infinite values and there positions.